In [1]:
import os
from glob import glob

import matplotlib.pylab as plt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plaidml.keras

plaidml.keras.install_backend()

In [2]:
from keras import applications
from keras.layers import Conv2D, Flatten, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Activation, Dense
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.preprocessing.image import (ImageDataGenerator, array_to_img,
                                       img_to_array, load_img)
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import cv2

In [3]:
# normal: 0
# error: 1
X = []
y = []
normal_dir = os.path.abspath(os.path.join("data", "正常"))
normal_path = glob(os.path.join(normal_dir, "*.bmp"))
for normal in normal_path:
    X.append(img_to_array(load_img(normal)))
    y.append(0)
spiderweb_dir = os.path.abspath(os.path.join("data", "蛛網"))
spiderweb_path = glob(os.path.join(spiderweb_dir, "*.bmp"))
for spiderweb in spiderweb_path:
    X.append(img_to_array(load_img(spiderweb)))
    y.append(1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
def adjustData(X, y):
    X = np.asarray(X)
    X = X.astype("float32")
    X = X / 255
    y = np.asarray(y)
    y = np_utils.to_categorical(y, num_classes=4)
    return X, y

In [5]:
adj_X_train, adj_y_train = adjustData(X_train, y_train)
adj_X_test, adj_y_test, = adjustData(X_test, y_test)

In [6]:
def SimpleCNN():
    model = Sequential()
    model.add(Conv2D(50, (3, 3), activation="relu", input_shape=(200, 200, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(50, (3, 3), activation="relu", input_shape=(200, 200, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(100, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(100, (3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(100, activation="relu"))
    model.add(Dense(100, activation="relu"))
    model.add(Dense(4, activation="softmax"))
    model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

    return model

In [7]:
%%time
model = SimpleCNN()
model.fit(adj_X_train, adj_y_train, batch_size=10, epochs=50, verbose=1)
model.save("model.h5")

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


Epoch 1/50


INFO:plaidml:Analyzing Ops: 105 of 373 operations complete


520/526 [============================>.] - ETA: 0s - loss: 0.3586 - acc: 0.8293

INFO:plaidml:Analyzing Ops: 105 of 373 operations complete


526/526 [==============================] - 25s 48ms/step - loss: 0.3576 - acc: 0.8303
Epoch 2/50
526/526 [==============================] - 3s 6ms/step - loss: 0.2952 - acc: 0.8798
Epoch 3/50
526/526 [==============================] - 3s 6ms/step - loss: 0.2524 - acc: 0.8992
Epoch 4/50
526/526 [==============================] - 3s 6ms/step - loss: 0.2365 - acc: 0.9140
Epoch 5/50
526/526 [==============================] - 3s 6ms/step - loss: 0.2449 - acc: 0.8959
Epoch 6/50
526/526 [==============================] - 3s 6ms/step - loss: 0.2229 - acc: 0.9135
Epoch 7/50
526/526 [==============================] - 3s 6ms/step - loss: 0.2085 - acc: 0.9230
Epoch 8/50
526/526 [==============================] - 3s 6ms/step - loss: 0.2176 - acc: 0.9259
Epoch 9/50
526/526 [==============================] - 3s 6ms/step - loss: 0.2053 - acc: 0.9268
Epoch 10/50
526/526 [==============================] - 3s 6ms/step - loss: 0.2089 - acc: 0.9297
Epoch 11/50
526/526 [==============================] - 3s 

In [8]:
model = load_model("model.h5")
score = model.evaluate(adj_X_test, adj_y_test, batch_size=10)
print("Acc:", score[1])

176/176 [==============================] - 6s 37ms/step
Acc: 0.9744318168271672


In [9]:
adj_X, _ = adjustData(X, y)
y_pred = model.predict_classes(adj_X)
count = 0
err_idx = []
for idx, pred in enumerate(y_pred):
    if pred != y[idx]:
        err_idx.append(idx)

In [10]:
for i in range(len(err_idx)):
    true = "normal" if y[err_idx[i]] == 1 else "spiderweb"
    pred = "normal" if y_pred[err_idx[i]] == 1 else "spiderweb"
    if true == "normal":
        cv2.imwrite("./false_pred/normal/true_%s_pred_%s-%d.png" %(true, pred, err_idx[i]), X_test[i])
    elif true == "spiderweb":
        cv2.imwrite("./false_pred/spiderweb/true_%s_pred_%s-%d.png" %(true, pred, err_idx[i]), X_test[i])

In [11]:
confusion_matrix(y, y_pred)

array([[495,   5],
       [  7, 195]], dtype=int64)